In [261]:
import pandas as pd
import re
from itertools import chain  

## SQL Read

In [262]:
# Leer el archivo .sql
with open('Query_polizas_gs.sql', 'r', encoding='utf-8') as file:
    sql_query = file.read()

## Summary Tranformation 

In [263]:
# Define una expresión regular para encontrar la descripcion general de la query.
patron = r'--Summary:(.*?)Sources:'

# Encuentra todas las coincidencias en el contenido del archivo
coincidencias = re.findall(patron, sql_query, re.DOTALL)

# Concatenar las coincidencias en una sola cadena de texto
summary_docs = '\n'.join(coincidencias).strip()
summary_docs = summary_docs.replace('-', ' ')

# Resaltar en negrita algunas palabras en summary_docs y agregar iconos al principio
summary_docs = summary_docs.replace('Created Date:','<i class="fas fa-calendar-alt icono"></i> <b style="color: #334559;">Created Date:</b>')
summary_docs = summary_docs.replace('Description:', '<i class="fa-solid fa-pen-to-square icono"></i> <b style="color: #334559;">Description:</b>')
summary_docs = summary_docs.replace('References:', '<i class="fas fa-book icono"></i> <b style="color: #334559;">References:</b>')   


## Source Transformation

In [264]:
# Define una expresión regular para encontrar la descripcion general de la query.
patron = r'--Sources:(.*?)Product 1:'

# Encuentra todas las coincidencias en el contenido del archivo
coincidencias = re.findall(patron, sql_query, re.DOTALL)

# Concatenar las coincidencias en una sola cadena de texto
source_docs = '\n'.join(coincidencias).strip()
source_docs  = source_docs .replace('-', ' ')
source_docs  = source_docs .replace(' ', '')


## Products Tranformation

In [265]:
# Define una expresión regular para encontrar la descripcion general de la query.
patron = r'(Product.*?)(?=Historical Versions:)'

# Encuentra todas las coincidencias en el contenido del archivo
coincidencias = re.findall(patron, sql_query, re.DOTALL)

# Concatenar las coincidencias en una sola cadena de texto
products_docs = '\n'.join(coincidencias).strip()
products_docs  = products_docs .replace('-', ' ')

# Se agrega un espacio delate de Product 1 para que quede alineado
products_docs  = products_docs .replace('Product 1', '  Product 1')

#Resaltar en negrita algunas palabras en summary_docs y agregar iconos al principio
products_docs = products_docs.replace('Product', '<i class="fa-solid fa-box-archive icono_2"></i> <b style="color: #334559;"> Product </b>')
products_docs = products_docs.replace('Description:', '   <i class="fa-solid fa-pen-to-square icono"></i> <b style="color: #334559;"> Description:</b>')
products_docs = products_docs.replace('Name:', '   <i class="fa-solid fa-hashtag icono"></i></i> <b style="color: #334559;"> Name:</b>')
products_docs = products_docs.replace('Type:', '   <i class="fa-solid fa-layer-group icono"></i> <b style="color: #334559;">Type:</b>')
products_docs = products_docs.replace('Process:', '   <i class="fa-solid fa-code-compare icono"></i> <b style="color: #334559;">Process:</b>')


## Hitorical Versions:

In [266]:
# Define una expresión regular para encontrar el detalle de las versiones historicas.
patron = r'--Historical Versions:(.*?)Step'

# Encuentra todas las coincidencias en el contenido del archivo
coincidencias = re.findall(patron, sql_query, re.DOTALL)

# Concatenar las coincidencias en una sola cadena de texto
versions_docs = '\n'.join(coincidencias).strip()
versions_docs  = versions_docs .replace('-', ' ')

# Cambiar el color de todo lo que esté entre paréntesis en versions_docs
versions_docs = re.sub(r'\((.*?)\)', r'<span style="color: #2e86c1;">(\1)</span>', versions_docs)


print(versions_docs)

  2025/01/01: <span style="color: #2e86c1;">(Emiliano Rigueiro)</span> created Query.
  2025/02/01: <span style="color: #2e86c1;">(Emiliano Rigueiro)</span> se elimino una de los ids considerados para las campañas.
  2025/01/01: <span style="color: #2e86c1;">(Emiliano Rigueiro)</span> created Query.
  2025/02/01: <span style="color: #2e86c1;">(Emiliano Rigueiro)</span> se elimino una de los ids considerados para las campañas. 
  2025/01/01: <span style="color: #2e86c1;">(Emiliano Rigueiro)</span> created Query.
  2025/02/01: <span style="color: #2e86c1;">(Emiliano Rigueiro)</span> se elimino una de los ids considerados para las campañas. 
  2025/01/01: <span style="color: #2e86c1;">(Emiliano Rigueiro)</span> created Query.
  2025/02/01: <span style="color: #2e86c1;">(Emiliano Rigueiro)</span> se elimino una de los ids considerados para las campañas.  
                                                                                                               
                        

## Comments Transformation

In [267]:
# Define una expresión regular para encontrar todo los comentarios
patron = r'(Step|LC)(.*?)--'

# Encuentra todas las coincidencias en el contenido del archivo
coincidencias = re.findall(patron, sql_query, re.DOTALL)

# Se guardar los comentarios en un df y en una lista. 
list_documentation = []
df_documentation = pd.DataFrame ()
df_documentation_html = pd.DataFrame ()

for coincidencia in coincidencias:
    list_documentation.append(str(coincidencia))


# Eliminar caracteres especiales de cada elemento de la lista
list_documentation = [re.sub(r"\)$", "", comentario, count=1) for comentario in list_documentation]
list_documentation = [re.sub(r"^\(", "", comentario, count=1) for comentario in list_documentation]
list_documentation = [re.sub(r"'", '', comentario) for comentario in list_documentation]

# Se guardan los comentarios en un DataFrame
df_documentation['comentarios'] = list_documentation
df_documentation_html['Comment'] = df_documentation['comentarios'].str.split(':').str[1]

# Se crea la campo Order que contiene el orden secuencial de los comentarios
df_documentation_html['Order'] = range(1, len(df_documentation) + 1)

# Se crea el campo Clase que identifica la clase a la que pertenece cada comentario
df_documentation_html['Clase'] = df_documentation['comentarios'].str.split(':').str[0]
df_documentation_html['Clase'] = df_documentation_html['Clase'].str.replace('_', '.')
df_documentation_html['Clase'] = df_documentation_html['Clase'].str.replace('  ', ' ')
df_documentation_html['Clase'] = df_documentation_html['Clase'].str.replace(',', ' ')

# Funcion para identificar a que linea pertenece cada comentario
def line_count(cadena, sql):
    line_tmp = []
    line = [i + 1 for i, linea in enumerate(sql.splitlines()) if cadena in linea]
    line_tmp.append(line)
    return line_tmp

# Se crea el campo line_number ejecuntando la función line_count
line_number = []

for elemento in df_documentation_html['Comment']:
    line_number.append(line_count(elemento, sql_query)) 

line_number = list(chain(*line_number))

df_documentation_html['Line Number'] = line_number  

# Reemplazar "LC" por "Line Comment" en la columna 'Comment'
df_documentation_html['Clase'] = df_documentation_html['Clase'].str.replace('LC', 'Line Comment', regex=False)

# Orden de las columnas del DataFrame
df_documentation_html = df_documentation_html[['Order', 'Clase', 'Comment', 'Line Number']]



In [268]:
print(df_documentation_html)

    Order           Clase                                            Comment  \
0       1         Step  1   Se obtienen todas las talta de HSBC a excepci...   
1       2  Line Comment     Se limitan las altas por id de estructura de ...   
2       3  Line Comment     Se limita el universo a la primera póliza (ce...   
3       4         Step  2   Proceso para obtener las altas de HSBC que se...   
4       5       Step  2.1   Se genera una temporal con las solicitudes ge...   
5       6  Line Comment     Se limita el universo solo a las campañas de ...   
6       7  Line Comment               Trae solamente las gestiones de ventas   
7       8       Step  2.2   Cruce con cotizaciones para obtener detalles ...   
8       9  Line Comment     Generarmo un flg para identifiar aquellas pól...   
9      10       Step  2.3   Se realiza el cruce con certificados utilizan...   
10     11  Line Comment     Generamos un flg para identifiar aquellas pól...   
11     12       Step  2.4   Se realiza u

## HTML Creation

In [269]:
import pandas as pd

# Convertir el DataFrame a HTML sin índice
table_html = df_documentation_html.to_html(index=False, escape=False)

# Agregar clases específicas a las columnas
table_html = table_html.replace("<th>Clase</th>", '<th class="Clase">Clase</th>')
table_html = table_html.replace("<th>Comment</th>", '<th class="Comment">Comment</th>')

# Agregar un título dentro de summary_docs
summary_docs = f"<h4 style='margin: 5px 0; font-size: 24px; color: #283747; border-bottom: 1.5px solid #ccc;'>Resumen de la consulta</h4>\n{summary_docs}"

# Dividir el contenido en líneas y agregar un icono a cada línea para source_docs
icono = '<i class="fa fa-sign-out icono"></i>'  
source_docs = source_docs.split('\n')
source_docs = '\n'.join([f"{icono} {line}" if line.strip() else line for line in source_docs])

# Dividir el contenido en líneas y agregar un icono a cada línea para source_docs
icono = '<i class="fa-solid fa-map-pin icono"></i>' 
versions_docs = versions_docs.split('\n')
versions_docs = '\n'.join([f"{icono} {line}" if line.strip() else line for line in versions_docs])

# Agregar un título dentro de source_docs
source_docs = f"<h4 style='margin: 5px 0; font-size: 24px; color: #283747; border-bottom: 1.5px solid #ccc;'>Fuentes consumidas</h4>\n{source_docs}"

# Agregar un título dentro de source_docs
products_docs = f"<h4 style='margin: 5px 0; font-size: 24px; color: #283747; border-bottom: 1.5px solid #ccc;'>Productos generados</h4>\n{products_docs}"

# Agregar un título dentro de versions_docs
versions_docs = f"<h4 style='margin: 5px 0; font-size: 24px; color: #283747; border-bottom: 1.5px solid #ccc;'>Historical Versions</h4>\n{versions_docs}"

# Agregar clases a las celdas de la columna "Comment"
for i in range(len(df_documentation_html['Comment'])):
    table_html = table_html.replace('<td>', '<td class="Comment">', 1)

# Definir el HTML completo con estilos CSS
html_string = f"""
<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Process Tabla de Datos</title>
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0-beta3/css/all.min.css">
    <style>
        body {{
            font-family: Arial, sans-serif;
            background-color: #2c2c2c;
            color: #fffcf2;
            text-align: left;
            padding: 25px;
        }}
        h2 {{
            color: #fffcf2;
        }}
        table {{
            margin: auto;
            border-collapse: collapse;
            background-color: #252422;
            color: #fffcf2;
            text-align: left;
            width: 100%; / Ajusta el ancho al 100% del contenedor
            table-layout: auto;  /* Ajusta el ancho de las columnas al contenido */
        }}
        th, td {{
            border-left: none;
            border-right: none;
            border-top: 0.5px solid #555555;
            border-bottom: 0.5px solid #555555;
            padding: 10px;
            text-align: left;
            
        }}
        th {{
            background-color: #444;
        }}
        .Clase {{
            min-width: 100px; /* Ajusta el ancho de la columna Ciudad */
        }}
        .summary {{
            font-family: Arial, sans-serif;
            font-size: 17px; /* Aumenta el tamaño de la letra */
            background-color: #fffcf2;
            color: #252422;
            text-align: left;
            margin: 10px 0 10px 0; /* Ajusta el margen superior e inferior */
            padding: 10px;
            border-radius: 10px; /* Redondea los bordes */
            padding-left: 20px; /* Añade sangría a cada elemento */ 
        }}
        .icono {{
            color: #2e86c1; /* Cambia el color del icono */
        
        }}
        .icono_2 {{
            color: #21618c; /* Cambia el color del icono */

        }}
        .source {{
            font-family: Arial, sans-serif;
            font-size: 17px; /* Aumenta el tamaño de la letra */
            background-color: #fffcf2;
            color: #343434;
            text-align: left;
            margin: 10px 0 10px 0; /* Ajusta el margen superior e inferior */
            padding: 10px;
            border-radius: 10px; /* Redondea los bordes */
            padding-left: 20px; /* Añade sangría a cada elemento */
        }}
    </style>
</head>
<body>
    <h2>Summary</h2>
    <pre class="summary">{summary_docs}</pre>
    <pre class="source">{source_docs}</pre>
    <pre class="source">{products_docs}</pre>
    <pre class="source">{versions_docs}</pre>
    <h2>Process Comments</h2>
    {table_html}
</body>
</html>
"""

# Guardar en un archivo
with open("tabla.html", "w", encoding="utf-8") as f:
    f.write(html_string)

print("Archivo HTML generado con éxito.")


Archivo HTML generado con éxito.
